# Livelike-PMEDMrcpp Bridge

In [1]:
%load_ext watermark
%watermark

Last updated: 2025-02-09T22:20:10.911559-05:00

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.32.0

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
import pathlib

import likeness_vitals
import livelike

import pmedm_legacy

%watermark -w
%watermark -iv

/Users/41q/miniforge3/envs/py312_pmedm_legacy/lib/python3.12/site-packages/likeness_vitals/__init__.py:12: FutureWarning: The ``likeness-vitals`` package is transitioning to a modular API for a cleaner top-level namespace. Classes, functions, and attributes that are currently available in the top-level will only be available within their home modules. Both the current and the updated API will be in place for at least two minor releases following the inclusion of this warning. The release at the time of this warning inclusion was ``v1.3.0``, so the current API will be stable until ``v1.5.0`` at the soonest, and the ``v2.0.0`` release will be the absolute deadline. See [https://code.ornl.gov/likeness/likeness-vitals/-/issues/32] for details on how to update dependant packages.
  warnings.warn(


Watermark: 2.5.0

likeness_vitals: 1.4.2
livelike       : 1.4.4
pmedm_legacy   : 0.0.6.dev17+g3f6adee



-------------------

## Get census API key

In [3]:
key = likeness_vitals.get_censusapikey(pathlib.Path("..", "..", "livelike", ""))

---------------
## Create PUMA class (`livelike.puma`)
### target PUMA - Knoxville City, TN

In [4]:
p = "4701603"
pup = livelike.puma(p, cache=True, censusapikey=key)

---------------------------
## Create a P-MEDM instance and solve

In [5]:
pmd = pmedm_legacy.PMEDM(
    pup.year,
    pup.wt,
    pup.est_ind,
    pup.est_g2,
    pup.est_g1,
    pup.se_g2,
    pup.se_g1,
    tr_iter=300,
    tr_tol=1e-4,
)
pmd.solve()

Using Sparse Hessian Preconditioner
Initializing PMEDM solver
Starting Trust Region Algorithm
 iter          fun     ||grad|| ||try step||      act red     pred red     radius inner iter
    0 -8.12572e-13     0.249819            1      -10.692      1.40111          1           0
    1 -8.12572e-13     0.249819         0.25     0.439019     0.444027       0.25           0
    2    -0.439019     0.222566          0.5     0.327436     0.544532        0.5           1
    3    -0.766454      3.14012          0.5    0.0454369     0.175763        0.5          12
    4    -0.811891      7.55848          0.5   -0.0928623     0.349079        0.5          22
    5    -0.811891      7.55848          nan          nan          nan      0.125          14
    6    -0.811891      7.55848          nan          nan          nan    0.03125          10
    7    -0.811891      7.55848          nan          nan          nan  0.0078125           9
    8    -0.811891      7.55848          nan          nan    

----------------------------
## Population Synthesis

In [6]:
syp = livelike.homesim.synthesize(
    pmd.pmedm_res["almat"],
    pup.est_ind,
    pup.est_g2,
    pup.sporder,
    nsim=1,
)
syp

,sim,geoid,count
h_id,,,
2015000013399,0,470930001001,1
2015000016188,0,470930001001,1
2015000020218,0,470930001001,2
2015000039553,0,470930001001,1
2015000048278,0,470930001001,2
...,...,...,...
2019HU1295854,0,470930071003,1
2019HU1303896,0,470930071003,2
2019HU1362049,0,470930071003,1


-------------------------------
## Diagnostics

In [7]:
dgs = pmedm_legacy.moe_fit_rate(
    pup.est_ind,
    pup.est_g2,
    pup.se_g2,
    pmd.pmedm_res["almat"],
)

In [8]:
dgs["Ycomp"]

,constraint,acs,pmedm,err,moe,in_moe
470930001001,population,2463,2.482511e+03,1.951072e+01,227.0,True
470930008001,population,1697,1.731241e+03,3.424083e+01,334.0,True
470930008002,population,574,6.015439e+02,2.754393e+01,235.0,True
470930008003,population,1336,1.379976e+03,4.397572e+01,448.0,True
470930009011,population,1965,1.897626e+03,6.737404e+01,299.0,True
...,...,...,...,...,...,...
470930070001,hht_nonfam_hhsize_7pm,0,9.719687e-10,9.719687e-10,12.0,True
470930070002,hht_nonfam_hhsize_7pm,0,5.536147e-09,5.536147e-09,12.0,True
470930071001,hht_nonfam_hhsize_7pm,0,1.202251e-18,1.202251e-18,12.0,True
470930071002,hht_nonfam_hhsize_7pm,0,7.657787e-19,7.657787e-19,12.0,True


In [9]:
dgs["moe_fit_rate"]

np.float64(0.9998638529611981)

### Check Population Totals

In [10]:
Ycomps = dgs["Ycomp"]
ycp = Ycomps[Ycomps["constraint"] == "population"]
ych = Ycomps[Ycomps["constraint"] == "housing_units"]

In [11]:
acs_totalpop = ycp["acs"].sum()
acs_totalpop

np.int64(178350)

In [12]:
pmedm_totalpop = ycp["pmedm"].sum()
pmedm_totalpop

np.float64(178786.87245946575)

In [13]:
# ACS 90% MOE 1327
pmedm_totalpop - acs_totalpop

np.float64(436.8724594657542)

In [14]:
acs_totalhu = ych["acs"].sum()
acs_totalhu

np.int64(87369)

In [15]:
pmedm_totalhu = ych["pmedm"].sum()
pmedm_totalhu

np.float64(87765.49277844286)

In [16]:
# ACS 90% MOE 1004
pmedm_totalhu - acs_totalhu

np.float64(396.49277844285825)

----------------------------------